In [1]:
import numpy as np
import os
import pandas as pd

In [ ]:
input_channel_mom = 'nov-2025/USDOnline-DashboardMk2_RChannelsMoM_Table.csv'
input_channel_yoy = 'nov-2025/USDOnline-DashboardMk2_RChannelsYoY_Table.csv'
output_channel = 'nov-2025/all_program_summaries_update.md'

In [ ]:
class ChannelAnalyzer:
    def __init__(self, mom_file, yoy_file):
        self.mom_file = mom_file
        self.yoy_file = yoy_file
        self.mom_df = None
        self.yoy_df = None
        self.merged_df = None
        self.program_totals = None
        self.summaries_mom = {}
        self.summaries_yoy = {}

    def load_data(self):
        self.mom_df = pd.read_csv(self.mom_file)
        self.yoy_df = pd.read_csv(self.yoy_file)
        print("MoM Data (head):")
        print(self.mom_df.head())
        print("YoY Data (head):")
        print(self.yoy_df.head())

    def clean_data(self):
        for df in [self.mom_df, self.yoy_df]:
            for col in df.columns:
                if 'Session' in col or 'Conversion' in col or 'Pageview' in col or 'difference' in col or 'Rate' in col:
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        print("Data cleaned.")

    def merge_data(self):
        self.merged_df = pd.merge(
            self.mom_df,
            self.yoy_df,
            on=['program_category', 'Channel'],
            suffixes=('_mom', '_yoy'),
            how='outer'
        )
        print("Merged Data (head):")
        print(self.merged_df.head())

    def recalc_percentages(self, df, suffix):
        if suffix == 'mom':
            session_diff = 'sessions_mom_difference'
            conv_diff = 'conversions_mom_difference'
            session_val = 'Session_mom'
            conv_val = 'Conversions_mom'
            prev_session = df[session_val] - df[session_diff]
            prev_conv = df[conv_val] - df[conv_diff]
            df['sessions_mom_percent'] = np.where(prev_session != 0, (df[session_diff] / prev_session) * 100, 0)
            df['conversions_mom_percent'] = np.where(prev_conv != 0, (df[conv_diff] / prev_conv) * 100, 0)
            df['conversion_rate_mom_percent'] = np.where(prev_session != 0,(np.where((prev_conv / prev_session) != 0, ((df[conv_val] / df[session_val]) - (prev_conv / prev_session)) * 100 / (prev_conv / prev_session), 0)),0)
        elif suffix == 'yoy':
            session_diff = 'sessions_yoy_difference'
            conv_diff = 'conversions_yoy_difference'
            session_val = 'Session_yoy'
            conv_val = 'Conversions_yoy'
            prev_session = df[session_val] - df[session_diff]
            prev_conv = df[conv_val] - df[conv_diff]
            df['sessions_yoy_percent'] = np.where(prev_session != 0, (df[session_val] - prev_session) / prev_session * 100, 0)
            df['conversions_yoy_percent'] = np.where(prev_conv != 0, (df[conv_val] - prev_conv) / prev_conv * 100, 0)
            df['conversion_rate_yoy_percent'] = np.where(prev_session != 0,(np.where((prev_conv / prev_session) != 0, ((df[conv_val] / df[session_val]) - (prev_conv / prev_session)) * 100 / (prev_conv / prev_session), 0)),0)
        return df

    def calculate_program_totals(self):
        totals = self.merged_df.groupby('program_category').agg({
            'Session_mom': 'sum',
            'sessions_mom_difference': 'sum',
            'Conversions_mom': 'sum',
            'conversions_mom_difference': 'sum',
            'Session_yoy': 'sum',
            'sessions_yoy_difference': 'sum',
            'Conversions_yoy': 'sum',
            'conversions_yoy_difference': 'sum',
        }).reset_index()
        totals['Conversion_Rate_mom'] = np.where(
            totals['Session_mom'] > 0,
            totals['Conversions_mom'] / totals['Session_mom'],
            0
        )
        totals['Conversion_Rate_yoy'] = np.where(
            totals['Session_yoy'] > 0,
            totals['Conversions_yoy'] / totals['Session_yoy'],
            0
        )
        totals = self.recalc_percentages(totals, 'mom')
        totals = self.recalc_percentages(totals, 'yoy')
        self.program_totals = totals
        print("Program totals:")
        print(self.program_totals.head())

    def get_top_channels(self, program, metric='Session_mom', n=3):
        prog_df = self.merged_df[self.merged_df['program_category'] == program].copy()
        prog_df = self.recalc_percentages(prog_df, 'mom')
        prog_df = self.recalc_percentages(prog_df, 'yoy')
        prog_df['Conversion_Rate_mom'] = np.where(
            prog_df['Session_mom'] > 0,
            prog_df['Conversions_mom'] / prog_df['Session_mom'],
            0
        )
        prog_df['Conversion_Rate_yoy'] = np.where(
            prog_df['Session_yoy'] > 0,
            prog_df['Conversions_yoy'] / prog_df['Session_yoy'],
            0
        )
        # Filter out channels containing "Unknown" (case-insensitive)
        prog_df = prog_df[~prog_df['Channel'].str.contains('Unknown', case=False, na=False)]
        top_channels = prog_df.sort_values(metric, ascending=False).head(n)
        return top_channels

    def generate_program_summary_yoy(self, program):
        row = self.program_totals[self.program_totals['program_category'] == program].iloc[0]
        summary = []
        traffic_trend = "increased" if row['sessions_yoy_difference'] > 0 else "decreased"
        conversion_trend = "improved" if row['conversions_yoy_difference'] > 0 else "declined"
        summary.append("# " + str(program) + " Performance Report\n")
        summary.append("### Executive Summary\nThe " + str(program) + " program has " + traffic_trend + " by " + str(abs(row['sessions_yoy_percent']))[:5] + "% year-over-year with " + str(int(row['Session_mom'])) + " total sessions. Conversions have " + conversion_trend + " by " + str(abs(row['conversions_yoy_percent']))[:5] + "% compared to last year, with an overall conversion rate of " + str(round(row['Conversion_Rate_mom']*100,2)) + "%.\n")
        summary.append("### Key Observations")
        summary.append("- Traffic has " + traffic_trend + " by " + str(abs(row['sessions_yoy_percent']))[:5] + "% year-over-year with " + str(int(row['Session_mom'])) + " total sessions.")
        top_channel = self.get_top_channels(program, 'Session_mom', 1)
        if not top_channel.empty:
            ch = top_channel.iloc[0]
            summary.append("- " + str(ch['Channel']) + " is the top traffic source with " + str(int(ch['Session_mom'])) + " sessions.")
        summary.append("- Conversions have " + conversion_trend + " by " + str(abs(row['conversions_yoy_percent']))[:5] + "% year-over-year with " + str(int(row['Conversions_mom'])) + " total conversions.\n")
        summary.append("### Key Performance Metrics\n")
        summary.append("#### Traffic\n- **Total sessions:** " + str(int(row['Session_mom'])) + " (YoY: " + str(int(row['sessions_yoy_difference'])) + ", " + str(round(row['sessions_yoy_percent'],2)) + "% | MoM: " + str(int(row['sessions_mom_difference'])) + ", " + str(round(row['sessions_mom_percent'],2)) + "%)\n")
        top_channels = self.get_top_channels(program, 'Session_mom', 3)
        summary.append("- **Top Channels:**")
        for i, (_, ch) in enumerate(top_channels.iterrows(), 1):
            summary.append("  " + str(i) + ". **" + str(ch['Channel']) + ":** " + str(int(ch['Session_mom'])) + " sessions (YoY: " + str(int(ch['sessions_yoy_difference'])) + ", " + str(round(ch['sessions_yoy_percent'],2)) + "% | MoM: " + str(int(ch['sessions_mom_difference'])) + ", " + str(round(ch['sessions_mom_percent'],2)) + "%)")
        summary.append("")
        summary.append("#### Conversions\n- **Total:** " + str(int(row['Conversions_mom'])) + " (YoY: " + str(int(row['conversions_yoy_difference'])) + ", " + str(round(row['conversions_yoy_percent'],2)) + "% | MoM: " + str(int(row['conversions_mom_difference'])) + ", " + str(round(row['conversions_mom_percent'],2)) + "%)\n")
        top_conv_channels = self.get_top_channels(program, 'Conversions_mom', 3)
        summary.append("- **Top Converting Channels:**")
        for i, (_, ch) in enumerate(top_conv_channels.iterrows(), 1):
            summary.append("  " + str(i) + ". **" + str(ch['Channel']) + ":** " + str(int(ch['Conversions_mom'])) + " conversions (YoY: " + str(int(ch['conversions_yoy_difference'])) + ", " + str(round(ch['conversions_yoy_percent'],2)) + "% | MoM: " + str(int(ch['conversions_mom_difference'])) + ", " + str(round(ch['conversions_mom_percent'],2)) + "%)")

        # FIX: Get the actual top 3 channels by conversion rate, not just the top traffic channels sorted by rate
        top_rate_channels = self.get_top_channels(program, 'Conversion_Rate_mom', 3)

        summary.append("\n#### Conversion Rates\n- **Overall:** " + str(round(row['Conversion_Rate_mom']*100,2)) + "% (YoY: " + str(round((row['conversion_rate_yoy_percent']),2)) + "% | MoM: " + str(round((row['conversion_rate_mom_percent']),2)) + "%)\n")
        summary.append("- **Top Channel Conversion Rates:**")
        for i, (_, ch) in enumerate(top_rate_channels.iterrows(), 1):
            # FIX: Use the correct field names (without '_difference' suffix)
            summary.append("  " + str(i) + ". **" + str(ch['Channel']) + ":** " + str(round(ch['Conversion_Rate_mom']*100,2)) + "% (YoY: " + str(round(ch['conversion_rate_yoy_percent'],2)) + "% | MoM: " + str(round(ch['conversion_rate_mom_percent'],2)) + "%)")
        return '\n'.join(summary)

    def generate_program_summary_mom(self, program):
        row = self.program_totals[self.program_totals['program_category'] == program].iloc[0]
        summary = []
        traffic_trend = "increased" if row['sessions_mom_difference'] > 0 else "decreased"
        conversion_trend = "improved" if row['conversions_mom_difference'] > 0 else "declined"
        summary.append("# " + str(program) + " Performance Report\n")
        summary.append("### Executive Summary\nThe " + str(program) + " program has " + traffic_trend + " by " + str(abs(row['sessions_mom_percent']))[:5] + "% month-over-month with " + str(int(row['Session_mom'])) + " total sessions. Conversions have " + conversion_trend + " by " + str(abs(row['conversions_mom_percent']))[:5] + "% compared to last month, with an overall conversion rate of " + str(round(row['Conversion_Rate_mom']*100,2)) + "%.\n")
        summary.append("### Key Observations")
        summary.append("- Traffic has " + traffic_trend + " by " + str(abs(row['sessions_mom_percent']))[:5] + "% month-over-month with " + str(int(row['Session_mom'])) + " total sessions.")
        top_channel = self.get_top_channels(program, 'Session_mom', 1)
        if not top_channel.empty:
            ch = top_channel.iloc[0]
            summary.append("- " + str(ch['Channel']) + " is the top traffic source with " + str(int(ch['Session_mom'])) + " sessions.")
        summary.append("- Conversions have " + conversion_trend + " by " + str(abs(row['conversions_mom_percent']))[:5] + "% month-over-month with " + str(int(row['Conversions_mom'])) + " total conversions.\n")
        summary.append("### Key Performance Metrics\n")
        summary.append("#### Traffic\n- **Total sessions:** " + str(int(row['Session_mom'])) + " (YoY: " + str(int(row['sessions_yoy_difference'])) + ", " + str(round(row['sessions_yoy_percent'],2)) + "% | MoM: " + str(int(row['sessions_mom_difference'])) + ", " + str(round(row['sessions_mom_percent'],2)) + "%)\n")
        top_channels = self.get_top_channels(program, 'Session_mom', 3)
        summary.append("- **Top Channels:**")
        for i, (_, ch) in enumerate(top_channels.iterrows(), 1):
            summary.append("  " + str(i) + ". **" + str(ch['Channel']) + ":** " + str(int(ch['Session_mom'])) + " sessions (YoY: " + str(int(ch['sessions_yoy_difference'])) + ", " + str(round(ch['sessions_yoy_percent'],2)) + "% | MoM: " + str(int(ch['sessions_mom_difference'])) + ", " + str(round(ch['sessions_mom_percent'],2)) + "%)")
        summary.append("")
        summary.append("#### Conversions\n- **Total:** " + str(int(row['Conversions_mom'])) + " (YoY: " + str(int(row['conversions_yoy_difference'])) + ", " + str(round(row['conversions_yoy_percent'],2)) + "% | MoM: " + str(int(row['conversions_mom_difference'])) + ", " + str(round(row['conversions_mom_percent'],2)) + "%)\n")
        top_conv_channels = self.get_top_channels(program, 'Conversions_mom', 3)
        summary.append("- **Top Converting Channels:**")
        for i, (_, ch) in enumerate(top_conv_channels.iterrows(), 1):
            summary.append("  " + str(i) + ". **" + str(ch['Channel']) + ":** " + str(int(ch['Conversions_mom'])) + " conversions (YoY: " + str(int(ch['conversions_yoy_difference'])) + ", " + str(round(ch['conversions_yoy_percent'],2)) + "% | MoM: " + str(int(ch['conversions_mom_difference'])) + ", " + str(round(ch['conversions_mom_percent'],2)) + "%)")

        # FIX: Get the actual top 3 channels by conversion rate, not just the top traffic channels sorted by rate
        top_rate_channels = self.get_top_channels(program, 'Conversion_Rate_mom', 3)

        summary.append("\n#### Conversion Rates\n- **Overall:** " + str(round(row['Conversion_Rate_mom']*100,2)) + "% (YoY: " + str(round((row['conversion_rate_yoy_percent']),2)) + "% | MoM: " + str(round((row['conversion_rate_mom_percent']),2)) + "%)\n")
        summary.append("- **Top Channel Conversion Rates:**")
        for i, (_, ch) in enumerate(top_rate_channels.iterrows(), 1):
            # FIX: Use the correct field names (without '_difference' suffix)
            summary.append("  " + str(i) + ". **" + str(ch['Channel']) + ":** " + str(round(ch['Conversion_Rate_mom']*100,2)) + "% (YoY: " + str(round(ch['conversion_rate_yoy_percent'],2)) + "% | MoM: " + str(round(ch['conversion_rate_mom_percent'],2)) + "%)")
        return '\n'.join(summary)

    def generate_all_program_summaries(self):
        for program in self.program_totals['program_category']:
            self.summaries_mom[program] = self.generate_program_summary_mom(program)
            self.summaries_yoy[program] = self.generate_program_summary_yoy(program)
        print("Generated summaries for all programs.")

    def write_summaries_to_markdown(self, filename):
        # Construct the a yoy and mom file name by splitting the original name and inserting the string
        name_part, extension_part = os.path.splitext(filename)
        yoy_filename = f"{name_part}_yoy{extension_part}"
        mom_filename = f"{name_part}_mom{extension_part}"
        with open(yoy_filename, 'w', newline='\n') as f:
            for program, summary in self.summaries_yoy.items():
                f.write(summary)
                f.write('\n\n---\n\n')
        with open(mom_filename, 'w', newline='\n') as f:
            for program, summary in self.summaries_mom.items():
                f.write(summary)
                f.write('\n\n---\n\n')
        print('Program summaries written to', yoy_filename)
        print('Program summaries written to', mom_filename)

    def get_cell_1_11(self):
        return self.program_totals.iloc[1, 11]

In [ ]:
analyzer = ChannelAnalyzer(input_channel_mom,input_channel_yoy)
analyzer.load_data()
analyzer.clean_data()
analyzer.merge_data()
analyzer.calculate_program_totals()
analyzer.generate_all_program_summaries()
analyzer.write_summaries_to_markdown(output_channel)

print(analyzer.summaries_yoy[list(analyzer.summaries_yoy.keys())[0]])
print(analyzer.program_totals.head())
print('Value at cell [1, 11]:', analyzer.get_cell_1_11())

MoM Data (head):
  program_category                          Channel  Session  \
0              MTS  Unknown - (not set) / (not set)   142473   
1              MED                   Organic Search    16224   
2            MSAAI                   Organic Search    12874   
3              MED                           Direct     5911   
4            MSAAI                           Direct     5830   

   sessions_mom_difference  Pageview  pageview_mom_difference  Conversions  \
0                 139091.0    113083                 110259.0          130   
1                   1982.0     18275                   1120.0           42   
2                    502.0     13584                   -421.0           43   
3                    863.0      6421                    182.0           29   
4                    670.0      6566                     81.0           35   

   conversions_mom_difference  Conversion_Rate  \
0                       127.0         0.000912   
1                         5.0